I implemented a few utils for scraping in the `RMPscraper.py` and `SchoolScraper.py` files. The code below scrapes Rate My Professors's MSU professor search page, collecting basic info about each of the professors:
- name
- prof_id: the numeric id that can be used to access their RMP page
- school
- department
- num_ratings: the number of ratings they have received

This data is then saved to a CSV file (called "test.csv" right now). 

In [1]:
from RMPscraper import RMPscraper
from SchoolScraper import get_card_info, save_card_info

In [2]:
school_id = 601 # MSU's school id

scraper = RMPscraper(base=f"https://www.ratemyprofessors.com/search/professors/{school_id}?q=*")
scraper.find_click(".CCPAModal__StyledCloseButton-sc-10x9kq-2") # Closes cookies popup
scraper.expand_all(max_iters=20)
page = scraper.source()
scraper.driver.quit()

cards = get_card_info(page)
save_card_info(cards, "test")

## TODO
- [ ] Write comment scraper for page given professor ID
    - [x] Implement
    - [ ] Fix scrolling behavior
- [ ] Scrape professor IDs for MSU
- [ ] Scrape comments on professors at MSU
- [ ] Choose other schools to get data from
- [ ] Repeat web scraping procedure ad infinitum

In [59]:
from ProfScraper import get_ratings, save_ratings

In [42]:
prof_id = 4183 # random school id

scraper = RMPscraper(base=f"https://www.ratemyprofessors.com/professor/{prof_id}", headless=False)
scraper.find_click(".CCPAModal__StyledCloseButton-sc-10x9kq-2") # Closes cookies popup
scraper.expand_all(max_iters=5)
page = scraper.source()
scraper.driver.quit()

ratings = get_ratings(page)
save_ratings(ratings, "test_ratings")

/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found
